# _Word Embeddings_

A **representação semântica distribuı́da** é baseada na hipótese distribucional que estabelece que o sentido de uma palavra é dado por seu contexto de ocorrência [2]. Esses vetores de palavras podem ser usados como recursos em uma variedade de aplicações, tais como: classificação de documentos [3], perguntas e respostas [4] e reconhecimento de entidade nomeada [5]. A representação de palavras como vetores contı́nuos tem uma longa história [6], [7], [8]). 

Muitos tipos diferentes de modelos foram propostos para estimar representações contı́nuas de palavras, incluindo a Análise Semântica Latente (do inglês, _Latent Semantic Analysis_ – **LSA**) e a Alocação Latente de Dirichlet (do inglês, _Latent Dirichlet Allocation_ – **LDA**). 

Já as representações distribuı́das de palavras aprendidas por redes neurais apresentam um desempenho significativamente superior ao LSA ao preservar regularidades lineares entre as palavras [1], [9]. Quanto ao LDA, sabe-se que ele é computacionalmente caro quando usado em grandes conjuntos de dados.

Neste notebook, faremos o treinamento do WordeVec em um córpus da Wikipédia no PT-BR.


In [8]:
#imports
import multiprocessing

from gensim.corpora import  WikiCorpus
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

In [9]:
#Save dataset/model
!mkdir data
!mkdir model
!mkdir "model/word2vec"

mkdir: cannot create directory ‘data’: File exists


## NNLM (_Neural Network Language Model_)

Uma arquitetura interessante de **NNLM** que foi apresentada por [10], é a que vetores de palavras são primeiro aprendidos usando uma rede neural com uma única camada oculta. Os vetores de palavras são, então, usados para treinar o NNLM. Assim, os vetores de palavra são aprendidos mesmo sem construir o NNLM completo. Mikolov et al. (2013) [1] estende diretamente essa arquitetura, focando apenas no primeiro passo onde os vetores de palavras são aprendidos usando um modelo simples. 

O objetivo é gerar vetores contendo números de tal forma que palavras similares de acordo com seus contextos estarão “próximas” no espaço vetorial, como ilustra a figura abaixo. Segundo Mikolov et al. (2013) [1], os vetores em cada lı́ngua foram projetados para 2 dimensões usando **PCA** e rotacionados manualmente para enfatizar a similaridade. Na figura temos palavras em inglês do lado direito e em espanhol do lado esquerdo.

<img src='images/word2vec_similaridade.png' width='600'>

Na célula seguinte, definiremos o caminho do córpus que utilizaremos para treinar o Word2Vec e onde o modelo será salvo.

O córpus que utilizaremos é um [dump da Wikipédia](https://www.dropbox.com/s/g2414vow56azzj2/wiki.pt-br.text.zip?dl=0). Baixe e salve na pasta "data".

In [10]:
#import data
import zipfile
from os.path import isfile, isdir

#O arquivo com a Wikipédia completa tem 1.8GB e pode ser baixado no link acima. O que utilizaremos tem apenas 307MB
#inp = "data/wiki.pt-br.text"
#dataset_filename = 'data/wiki.pt-br.text.zip'
outp = "model/word2vec/wiki.pt-br.word2vec.model"

inp = "data/wiki.pt-br_part.text"
dataset_filename = 'data/wiki.pt-br_part.text.zip'
dataset_folder_path = 'data'

with zipfile.ZipFile(dataset_filename) as zip_ref:
    zip_ref.extractall(dataset_folder_path)

## Word2Vec

Partindo da premissa de que técnicas básicas como contagem de n-gramas já estão em seu limite, Mikolov et al. (2013) [1] propõe a utilização de modelos de linguagem baseados em redes neurais para modelar representações distribuídas de palavras. O principal objetivo das técnicas propostas por Mikolov et al. (2013) [1] é aprender vetores de palavras de alta qualidade, a partir de enormes conjuntos de dados com bilhões de palavras. De maneira surpreendente, verificou-se que a similaridade das representações de palavras vai além das simples regularidades sintáticas. Dentro de um espaço de dimensões vetoriais, usando uma simples operação algébrica nos vetores de palavras, foi mostrado por exemplo que:

> vetor(**rei**) - vetor(**homem**) + vetor(**mulher**) = vetor que está próximo da representação vetorial da palavra **rainha**.

Mikolov et al. (2013) [1] propõe duas arquiteturas de modelos para a aprendizagem de representações distribuı́das de palavras que tentam minimizar a complexidade computacional: o modelo _Continuous Bag-of-Words_ (CBOW) e o modelo Skip-gram.

* **CBOW** – No CBOW, a arquitetura é semelhante à do NNLM _feedforward_, onde a camada escondida não-linear é removida e a camada de projeção é compartilhada para todas as palavras (não apenas a matriz de projeção). Assim, todas as palavras são projetadas na mesma posição. Essa arquitetura é chamada de modelo de saco de palavras (_bag of words_), pois a ordem das palavras não influencia a projeção. O CBOW usa representação distribuı́da contı́nua do contexto. A arquitetura do modelo é mostrada na figura abaixo, na qual pode-se observar que a matriz de pesos entre a entrada e a camada de projeção é compartilhada para todas as posições de palavras (da mesma maneira que no NNLM).
    
    
* **Skip-gram** – A arquitetura do Skip-gram é semelhante à do CBOW, mas em vez de prever a palavra atual com base no contexto, Skip-gram tenta maximizar a classificação de uma palavra com base em outra da mesma sentença. Mais precisamente, usa-se cada palavra atual como uma entrada para um classificador log-linear para prever palavras dentro de um intervalo anterior e posterior à palavra atual. O aumento do intervalo melhora a qualidade dos vetores de palavra resultantes, mas também aumenta a complexidade computacional. A distância entre uma palavra do contexto e a palavra atual indica o grau de relação entre elas. Quanto mais distante, menos relacionada estará à palavra atual, podendo receber pesos menores.

<img src='images/CBOW_Skip-Gram.png' width='500'>

### Parâmetros

Na próxima célula de código, definimos os seguintes parâmetros:

* **sg**: define o algoritmo de treinamento. Por padrão, o CBOW é usado (sg = 0). O outro é o skip-gram (sg = 1).

* **size**: dimensionalidade dos vetores.

* **window**: é a quantidade de palavras anteriores e posteriores à palavra alvo.

* **LineSentence**: Interpreta uma string ou arquivo. Cada linha é uma sentença.

* **min_count**: ignore as palavras com frequência total inferior a **min_count**.

* **max_vocab_size**: Limite a RAM durante a construção do vocabulário; se houver mais palavras únicas do que **max_vocab_size**, ocorre a poda os infrequentes. Cada 10 milhões de tipos de palavras precisam de cerca de 1GB de RAM.

* **sample**: limiar para configurar quais palavras de maior frequência são aleatoriamente reduzidas; O padrão é 1e-3, o intervalo útil é (0, 1e-5).

* **workers**: parâmetro que indica quantos cores da máquina serão utilizados para o treinamento.

* **hs**: se 1, softmax hierárquico será usado para o treinamento do modelo. Se definido como 0 (padrão), e existir amostragem negativa, esse recurso será utilizado.

* **negative**: se > 0, será utilizada amostragem negativa. O valor indica quantas "palavras de ruído" devem ser consideradas (normalmente entre 5 a 20). Se **negative** configurado para 0, não é utilizada a amostragem negativa.

* **cbow_mean**: se 0, usa a soma dos vetores das palavras de contexto. Se 1 (padrão), usa a média. Aplica-se apenas quando cbow é utilizado.

* **hashfxn**: função hash para inicializar os pesos aleatoriamente.

* **iter**: número de iterações (épocas) sobre o córpus. O padrão é 5.


In [11]:
#train model
%time model = Word2Vec(LineSentence(inp), size=400, window=5, min_count=5, workers=multiprocessing.cpu_count())

CPU times: user 18min 44s, sys: 4.03 s, total: 18min 48s
Wall time: 5min 13s


* **init_sims(replace=True)**: Normaliza o modelo para não demandar tanta memória.

In [12]:
# trim unneeded model memory = use (much) less RAM
model.init_sims(replace=True)

Salva o modelo no caminho especificado em outp

In [13]:
model.save(outp)

## Leituras

Sugiro as seguintes leituras complementares sobre o Word2Vec.

* A really good [conceptual overview](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/) of word2vec from Chris McCormick 
* [First word2vec paper](https://arxiv.org/pdf/1301.3781.pdf) from Mikolov et al.
* [NIPS paper](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) with improvements for word2vec also from Mikolov et al.
* An [implementation of word2vec](http://www.thushv.com/natural_language_processing/word2vec-part-1-nlp-with-deep-learning-with-tensorflow-skip-gram/) from Thushan Ganegedara
* TensorFlow [word2vec tutorial](https://www.tensorflow.org/tutorials/word2vec)
* [Deep Learning com Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html) por Gensim

## Referências

[1] [Efficient estimation of word representations in vector space.](https://arxiv.org/abs/1301.3781)

[2] [Multimodal distributional semantics.](https://www.jair.org/media/4135/live-4135-7609-jair.pdf)

[3] [Machine learning in automated text categorization.](http://delivery.acm.org/10.1145/510000/505283/p1-sebastiani.pdf?ip=200.137.216.145&id=505283&acc=ACTIVE%20SERVICE&key=344E943C9DC262BB%2E0ACEC6856BE69272%2E4D4702B0C3E38B35%2E4D4702B0C3E38B35&__acm__=1518017578_f5561e072809aadaea8bb04a71a5b21c)

[4] [Quantitative evaluation of passage retrieval algorithms for question answering.](http://delivery.acm.org/10.1145/870000/860445/p41-tellex.pdf?ip=200.137.216.145&id=860445&acc=ACTIVE%20SERVICE&key=344E943C9DC262BB%2E0ACEC6856BE69272%2E4D4702B0C3E38B35%2E4D4702B0C3E38B35&__acm__=1518017629_0f0d78efd0501b7ad05a74e586cd7ef8)

[5] [Word representations: a simple and general method for semi-supervised learning.](http://delivery.acm.org/10.1145/1860000/1858721/p384-turian.pdf?ip=200.137.216.145&id=1858721&acc=OPEN&key=344E943C9DC262BB%2E0ACEC6856BE69272%2E4D4702B0C3E38B35%2E6D218144511F3437&__acm__=1518017678_cec1b87c9c6e3f9ccd8e61f591acaa26)

[6] [Distributed representations.](https://web.stanford.edu/~jlmcc/papers/PDP/Chapter3.pdf)

[7] [Learning internal representations by back-propagating errors. Parallel Distributed Processing: Explorations in the Microstructure of Cognition](http://lia.disi.unibo.it/Courses/SistInt/articoli/nnet1.pdf)

[8] [Finding structure in time.](http://onlinelibrary.wiley.com/doi/10.1207/s15516709cog1402_1/epdf)

[9] [Combining heterogeneous models for measuring relational similarity.](http://www.aclweb.org/anthology/N13-1120)

[10] [Neural network based language models for highly inflective languages.](http://ieeexplore.ieee.org/abstract/document/4960686/)